In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import re
import nltk
import string
import nlp_utils
import contractions # I'll -> I will 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer

In [3]:
df = pd.read_csv('train.csv', encoding='latin-1')

In [4]:
df.columns =['target', 'id', 'date', 'flag', 'user', 'comment_text']
df

,target,id,date,flag,user,comment_text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   target        1599999 non-null  int64 
 1   id            1599999 non-null  int64 
 2   date          1599999 non-null  object
 3   flag          1599999 non-null  object
 4   user          1599999 non-null  object
 5   comment_text  1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [6]:
df.isnull().sum()

target          0
id              0
date            0
flag            0
user            0
comment_text    0
dtype: int64

In [7]:
df['target'].value_counts()

4    800000
0    799999
Name: target, dtype: int64

### Text Pre-processing

In [8]:
df['comment_text'][12]

'@octolinz16 It it counts, idk why I did either. you never talk to me anymore '

In [9]:
import re
import string

In [10]:
# Removing special characters
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x : re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
remove_n = lambda x: re.sub("\n", " ", x)
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]', r' ', x)
df['comment_text'] = df['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

In [11]:
Target_comment_df = df.loc[:, ['id','comment_text', 'target']]

In [12]:
Target_comment_df

,id,comment_text,target
0,1467810672,is upset that he can t update his facebook by ...,0
1,1467810917,kenichan i dived many times for the ball man...,0
2,1467811184,my whole body feels itchy and like its on fire,0
3,1467811193,nationwideclass no it s not behaving at all ...,0
4,1467811372,kwesidei not the whole crew,0
...,...,...,...
1599994,2193601966,just woke up having no school is the best fee...,4
1599995,2193601969,thewdb com very cool to hear old walt interv...,4
1599996,2193601991,are you ready for your mojo makeover ask me f...,4
1599997,2193602064,happy birthday to my boo of alll time tup...,4


In [13]:
import wordcloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords

In [14]:
# Visualizing dataset using wordcloud
def wordcloud(df, label):
    subset = df[df[label] == 0]
    text = subset.comment_text.values
    wc = WordCloud(background_color = "black", max_words = 2000)
    wc.generate(" ".join(text))
    plt.figure(figsize = (20, 20))
    plt.subplot(221)
    plt.axis("off")
    plt.title("Words frequented in {}".format(label), fontsize = 20)
    plt.imshow(wc.recolor(colormap = 'gist_earth', random_state = 244), alpha = 0.98)

In [15]:
# wordcloud(Target_comment_df, 'target')

#### ML

- balancing target column in df

In [16]:
Target_comment_df['target'].value_counts()

4    800000
0    799999
Name: target, dtype: int64

In [17]:
Target_comment_balanced_4 = Target_comment_df[Target_comment_df['target'] == 4].iloc[0:7999, :]

In [18]:
Target_comment_balanced_0 = Target_comment_df[Target_comment_df['target'] == 0].iloc[0:7999, :]

In [19]:
Target_comment_balanced = pd.concat([Target_comment_balanced_4, Target_comment_balanced_0])

In [20]:
Target_comment_balanced['target'].value_counts()

4    7999
0    7999
Name: target, dtype: int64

####  ML parts

In [21]:
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, fbeta_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import ngrams, bigrams, trigrams

In [24]:
def cv_tf_train_test(dataframe, label, vectorizer, ngram):
    
    # Split the data into X and y data sets
    X = dataframe.comment_text
    y = dataframe[label]
    
    # Split data in train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 50)
    
    # Using vectorizer to remove stopwords
    cv1 = vectorizer(ngram_range = (ngram), stop_words = 'english')
    
    # Transforming x-train and x-test
    X_train_cv1 = cv1.fit_transform(X_train)
    X_test_cv1 = cv1.transform(X_test)
    
    ### ML models
    
    ## logistic regression 
    lr = LogisticRegression()
    lr.fit(X_train_cv1, y_train)
    
    ## k-nearest neighbours 
    knn = KNeighborsClassifier(n_neighbors = 5)
    knn.fit(X_train_cv1, y_train)
    
    ## Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(X_train_cv1, y_train)
    
    ## Multinomial Naive Bayes
    mnb = MultinomialNB()
    mnb.fit(X_train_cv1, y_train)
    
    ## Support vector machine
    svm_model = LinearSVC()
    svm_model.fit(X_train_cv1, y_train)
    
    ## Random forest
    randomforest = RandomForestClassifier(n_estimators = 100, random_state = 50)
    randomforest.fit(X_train_cv1, y_train)
    
    """
     f1_score_data = {'F1 Score':[f1_score(lr.predict(X_test_cv1), y_test), 
                                f1_score(knn.predict(X_test_cv1), y_test), 
                                f1_score(bnb.predict(X_test_cv1), y_test), 
                                f1_score(mnb.predict(X_test_cv1), y_test), 
                                f1_score(svm_model.predict(X_test_cv1), y_test),
                                f1_score(randomforest.predict(X_test_cv1), y_test)]}
    """
    f1_score_data = {'F1 Score':[f1_score(lr.predict(X_test_cv1), y_test, pos_label = 4), 
                                f1_score(knn.predict(X_test_cv1), y_test, pos_label = 4), 
                                f1_score(bnb.predict(X_test_cv1), y_test, pos_label = 4), 
                                f1_score(mnb.predict(X_test_cv1), y_test, pos_label = 4), 
                                f1_score(svm_model.predict(X_test_cv1), y_test, pos_label = 4),
                                f1_score(randomforest.predict(X_test_cv1), y_test, pos_label = 4)]}
    ## Saving f1 score results into a dataframe
    df_f1 = pd.DataFrame(f1_score_data, index = ['LR','KNN','BNB', 'MNB','SVM','RF'])
    return df_f1
    
    

#### Evaluating model performance using evaluation metric

In [25]:
Target_comment_cv = cv_tf_train_test(Target_comment_balanced, 'target', TfidfVectorizer, (1, 1))
Target_comment_cv.rename(columns = {'F1 Score' : 'F1 Score(Target)'}, inplace = True)
Target_comment_cv

,F1 Score(Target)
LR,0.732245
KNN,0.547905
BNB,0.698588
MNB,0.705017
SVM,0.720921
RF,0.716732


In [28]:
X = Target_comment_balanced.comment_text
y = Target_comment_balanced['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

tfv = TfidfVectorizer(ngram_range = (1,1), stop_words = 'english')
    

X_train_fit = tfv.fit_transform(X_train)
X_test_fit = tfv.transform(X_test)
    
randomforest = RandomForestClassifier(n_estimators = 100, random_state = 50)
randomforest.fit(X_train_fit, y_train)
randomforest.predict(X_test_fit)

array([0, 4, 0, ..., 0, 4, 4], dtype=int64)

In [29]:
#Testing model

c1 = ["I killed the insect and ate it"]
c1_vect = tfv.transform(c1)
randomforest.predict_proba(c1_vect)[:,1]

array([0.2602204])

In [33]:
#Testing model

c2 = ["I love dogs"]
c2_vect = tfv.transform(c2)
randomforest.predict_proba(c2_vect)[:,1]

array([0.91])